
# Lab 9: Build a Log Aggregator

In this lab, you will create your own log generator, build a command-line utility that scans log files, summarizes their contents, and provides insight into system behavior. Data structures to track log message levels such as `INFO`, `WARNING`, `ERROR`, and `CRITICAL`.

This lab reinforces:
- File I/O
- Pattern recognition (regex)
- Dictionaries and counters
- Functions and modularity
- Optional: CLI arguments, logging



## Part 1: Create Log files (20%)
Using the the following example log format below create a **python file** that will log errors In a structured tree format 

You will find examples in the folder called Logs that you can use to build your program.

Remember set of logs should have a varied levels of log entries (`INFO`, `WARNING`, `ERROR`, `CRITICAL`) and tailored message types for different service components.
You must create 5 structured logs here are some examples:

    sqldb
    ui
    frontend.js
    backend.js
    frontend.flask
    backend.flask

You may use chat GPT to create sample outputs NOT THE LOGS. IE:

    System failure
    Database corruption
    Disk failure detected
    Database corruption


In [ ]:
# Paste your python file here 
# don't forget to upload it with your submission




### Example Log Format

You will work with logs that follow this simplified structure:

```
2025-04-11 23:20:36,913 | my_app | INFO | Request completed
2025-04-11 23:20:36,914 | my_app.utils | ERROR | Unhandled exception
2025-04-11 23:20:36,914 | my_app.utils.db | CRITICAL | Disk failure detected
```


In [ ]:
import logging
import random
import time
from datetime import datetime


options = ["INFO", "WARNING", "ERROR", "CRITICAL"]
possible_messages = [ " System failure", "Database corruption",
    "Disk failure detected", "Database corruption"]

file_path_base = "logGLfolder\\log"
file_path_postfix_core = "_core"
file_path_postfix_utils = "_utils"
file_path_second_postfix = "_db"

postfix_group = [file_path_postfix_core, file_path_postfix_utils,0]

second_postfix_group = [file_path_second_postfix,0]


file_path_extension = ".log"




now = datetime.now()

def getCurrentTime():
    return now.strftime("%Y-%m-%d %H:%M:%S")

def getRandomLogLevel():
    return random.choice(options)

def getRandomMessage():
    return random.choice(possible_messages)




for i in range(100):
    log_level = getRandomLogLevel()
    message = getRandomMessage()

    postfix = random.choice(postfix_group)
    second_postfix = random.choice(second_postfix_group)


    

    if postfix == 0:
        file_path = file_path_base + file_path_extension

        log_message = f"{getCurrentTime()} | {file_path} | {log_level} | {message}\n"

        with open(file_path, "a") as file:
                file.write(log_message)
    else:
        if second_postfix == 0:
            file_path = file_path_base + postfix + file_path_extension


            log_message = f"{getCurrentTime()} | {file_path} | {log_level} | {message}\n"
        

            # can be done recursively - split by the underscore remove the last item and recombine into a string
            with open(file_path, "a") as file:
                file.write(log_message)

            file_path = file_path_base + file_path_extension

            with open(file_path, "a") as file:
                file.write(log_message)


        else:
            file_path = file_path_base + postfix + second_postfix + file_path_extension

            log_message = f"{getCurrentTime()} | {file_path} | {log_level} | {message}\n"
        
            with open(file_path, "a") as file:
                file.write(log_message)

            file_path = file_path_base + postfix + file_path_extension

            with open(file_path, "a") as file:
                file.write(log_message)

            file_path = file_path_base  + file_path_extension

            with open(file_path, "a") as file:
                file.write(log_message)






## Part 2: Logging the Log File (40%)
    New File
### Part 2a: Read the Log File (see lab 7) (10%)


Write a function to read the contents of a log file into a list of lines. Handle file errors gracefully.

### Part 2b: Parse Log Lines (see code below if you get stuck) (10%)

Use a regular expression to extract:
- Timestamp
- Log name
- Log level
- Message

### Part 2c: Count Log Levels (20%)

Create a function to count how many times each log level appears. Store the results in a dictionary. Then output it as a Json File
You may pick your own format but here is an example. 
```python
{
    "INFO": 
    {
        "Request completed": 42, 
        "Heartbeat OK": 7
    }

    "WARNING":
    {
        ...
    }
}

```


In [ ]:
# Paste your python file here don't for get to upload it with your submission

In [ ]:
# Paste your python file here 
# don't forget to upload it with your submission


## Step 3: Generate Summary Report (40%)
    New File
### Step 3a (20%):
 Develop a function that continuously monitors your JSON file(s) and will print a real-time summary of log activity. It should keep count of the messages grouped by log level (INFO, WARNING, ERROR, CRITICAL) and display only the critical messages. (I.e. If new data comes in the summary will change and a new critical message will be printed)
 - note: do not reprocess the entire file on each update.  

### Step 3a: Use a Matplotlib (Lecture 10) (20%)
Develop a function that continuously monitors your JSON file(s) and will graph in real-time a bar or pie plot of each of the errors.  (a graph for each log level). 
- The graph should show the distribution of log messages by level  (INFO, WARNING, ERROR, CRITICAL)  


### Critical notes:
- Your code mus use Daemon Threads (Lecture 14)
- 3a and 3b do not need to run at the same time. 


In [ ]:
# Paste your python file here 
# don't forget to upload it with your submission

In [1]:
import lab9_generator
import lab9_parser
import lab9Graph
import threading
import sys
import json

# all the threads are to be ran from here

dictionary = {"INFO": {}, "WARNING": {}, "ERROR": {}, "CRITICAL": {}}

def main():
    t1 = threading.Thread(target=lab9_generator.logGenerator)
    t2 = threading.Thread(target=lab9_parser.logParser, args=("logGLfolder\\log.log",dictionary))
    t3 = threading.Thread(target=lab9Graph.dict_listener, args=(dictionary))
    
    t2.start()
    t3.start()
    t1.start()
    

    # timout added since program was being hung 
    t2.join(timeout=4)
    t3.join(timeout=10)
    t1.join()


    print("Done!")
    print("Final dictionary:", dictionary)

    with open("logGLfolder\\log_summary.json", "w") as json_file:
        json.dump(dictionary, json_file, indent=4)

    # this does not work at all, cry face
    lab9_parser.exitTimer()

   
    

if __name__ == "__main__":
    main()

Exception in thread Thread-5 (dict_listener):
Traceback (most recent call last):
  File "c:\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Python312\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
TypeError: dict_listener() takes 1 positional argument but 4 were given


Monitoring log file: logGLfolder\log.log
Critical log entry detected!

Time: 2025-04-27 18:21:53  CRITICAL Found at  logGLfolder\log_utils_db.log  with message:  Database corruption

Critical log entry detected!

Time: 2025-04-27 18:21:53  CRITICAL Found at  logGLfolder\log_core.log  with message:  Database corruption

Critical log entry detected!

Time: 2025-04-27 18:21:53  CRITICAL Found at  logGLfolder\log.log  with message:  Database corruption

Critical log entry detected!

Time: 2025-04-27 18:21:53  CRITICAL Found at  logGLfolder\log.log  with message:  Disk failure detected

Critical log entry detected!

Time: 2025-04-27 18:21:53  CRITICAL Found at  logGLfolder\log_core.log  with message:  Database corruption

Critical log entry detected!

Time: 2025-04-27 18:21:53  CRITICAL Found at  logGLfolder\log_core.log  with message:  Disk failure detected

Critical log entry detected!

Time: 2025-04-27 18:21:53  CRITICAL Found at  logGLfolder\log_utils.log  with message:  Database corrup

NameError: name 'exit' is not defined

Critical log entry detected!

Time: 2025-04-27 18:22:46  CRITICAL Found at  logGLfolder\log.log  with message:  Disk failure detected

Critical log entry detected!

Time: 2025-04-27 18:22:46  CRITICAL Found at  logGLfolder\log_utils.log  with message:  Disk failure detected

Critical log entry detected!

Time: 2025-04-27 18:22:46  CRITICAL Found at  logGLfolder\log.log  with message:  Database corruption

Critical log entry detected!

Time: 2025-04-27 18:22:46  CRITICAL Found at  logGLfolder\log_utils.log  with message:   System failure

Critical log entry detected!

Time: 2025-04-27 18:22:46  CRITICAL Found at  logGLfolder\log_utils_db.log  with message:   System failure

Critical log entry detected!

Time: 2025-04-27 18:22:46  CRITICAL Found at  logGLfolder\log_core_db.log  with message:  Database corruption

Critical log entry detected!

Time: 2025-04-27 18:22:46  CRITICAL Found at  logGLfolder\log.log  with message:  Disk failure detected

Critical log entry detected!

Time: 2025-0